## Model context class

Wraps shadie simulation code within a `Model` class context wrapper so that it can be formatted and checked on exit. We can subclass the `Model` class to provide different defaults, for example, `ModelNucleotide` or `ModelTreeseq`. The main `Model` class should be generic with the expectation that the user will specify more in this script.

In [1]:
import shadie
shadie.set_loglevel("DEBUG")

2021-08-24 14:22:52.426 | WARNING  | shadie.base.mutations:__init__:73 - shadie workflow strongly advises against the use of neutral mutations in SLiM simulation unless Chromosome parameter NSsites = False (default = True)


### Simple example

In [4]:
with shadie.Model() as model:
    
    # define a simple chromosome of Elements
    chrom = shadie.chromosome.default()
    
    # create init block
    model.initialize(chrom)
    
    # create simulation
    #model.early(time=1, scripts="sim.addSubpop('p1', 10);")
    #model.late(time=1000, scripts="sim.outputFull();")

02:23 | DEBUG   | initialize      | initializing Model


AssertionError: You must call initialize() from within Model context

In [3]:
# print(model.script)

In [4]:
model.run()

In [5]:
# print(model.stdout)

### A more complex example

In [6]:
with shadie.Model() as model:
    
    # define mutation types
    m0 = shadie.mtype(0.5, 'n', 2.0, 1.0)
    m1 = shadie.mtype(0.5, 'g', 3.0, 1.0)
    
    # define elements types
    e0 = shadie.etype([m0, m1], [1, 2])
    e1 = shadie.etype([m1], [1])
    
    # design chromosome of elements
    chrom = shadie.chromosome.explicit({
        (500, 1000): e0,
        (1500, 2000): e1,
    })
    
    # create the init block
    model.initialize(chromosome=chrom, constants={"K": 500})

    # create simulation using custom shadie elements
    # that append to early, late, and repro code blocks
    #model.complex.alternation_of_generations()
    
    # create simulation manually
    model.early(time=1, scripts="sim.addSubpop('p1', 10);")
    model.early(time=None, scripts="p1.fitnessScaling = K / p1.individualCount;")
    model.late(time=2000, scripts="sim.outputFull();")
    model.late(
        time=None,
        scripts=[
            "inds = p1.individuals;",
            "catn(sim.generation + ': ' + size(inds) + ' (' + max(inds.age) + ')');"
        ]
    )

11:55 | DEBUG   | initialize      | initializing Model
11:55 | DEBUG   | __exit__        | exiting Model


In [7]:
# print(model.script)

In [8]:
model.run()

In [9]:
# print(model.stdout)

### Alternation of generations (manually)

In [21]:
# to add arbitrary SLIM code we would need to expand curly
# brackets to be double-curly-brackets.
repro = """
{
	g_1 = genome1; //defined here to simply code downstream
	g_2 = genome2;
	
	for (meiosisCount in 1:5) //each diploid individual undergoes meiosis 5x
	{
		if (individual.sex == "M")
		{
			// drawBreakpoints generates the same breakpoints that internal SLiM would generate
			//uses recomobination rate set for the moodel
			breaks = sim.chromosome.drawBreakpoints(individual);
			//resulting haploids are added to p2:
			s_1 = p2.addRecombinant(g_1, g_2, breaks, NULL, NULL, NULL, "M");
			//addRecombinant(leaves genomes and breakpoints NULL to avoid making second chromosome copy)
			s_2 = p2.addRecombinant(g_2, g_1, breaks, NULL, NULL, NULL, "M");
			
			breaks = sim.chromosome.drawBreakpoints(individual);
			s_3 = p2.addRecombinant(g_1, g_2, breaks, NULL, NULL, NULL, "M");
			s_4 = p2.addRecombinant(g_2, g_1, breaks, NULL, NULL, NULL, "M");
		}
		else if (individual.sex == "F")
		{
			breaks = sim.chromosome.drawBreakpoints(individual);
			if (runif(1) <= 0.5)
				e = p2.addRecombinant(g_1, g_2, breaks, NULL, NULL, NULL, "F");
			else
				e = p2.addRecombinant(g_2, g_1, breaks, NULL, NULL, NULL, "F");
		}
	}
}
"""

In [22]:
with shadie.Model() as model:
    
    # define mutation types
    m0 = shadie.mtype(0.5, 'n', 2.0, 1.0)
    m1 = shadie.mtype(0.5, 'g', 3.0, 1.0)
    
    # define elements types
    e0 = shadie.etype([m0, m1], [1, 2])
    e1 = shadie.etype([m1], [1])
    
    # design chromosome of elements
    chrom = shadie.chromosome.explicit({
        (500, 1000): e0,
        (1500, 2000): e1,
    })
    
    # create the init block
    model.initialize(
        chromosome=chrom, 
        constants={"K": 500},
        scripts=["initializeSex('A');"]
    )

    # create simulation manually
    model.early(
        time=1,
        scripts=[
            "sim.addSubpop('p1', K);"
            "sim.addSubpop('p2', 0);"
        ],
    )
    
    model.reproduction(
        population='p1',
        scripts=repro,
    )
    
    model.reproduction(
        population='p2',
        scripts=[],
    )
    
    # ...

11:58 | DEBUG   | initialize      | initializing Model
11:58 | DEBUG   | __exit__        | exiting Model


In [19]:
# model.script